In [4]:
import numpy as np
import matplotlib.pyplot as plt
from src.data_util import load_augmented_example, load_preprocess_examples
from src.evaluation import DAP_SAP_MAP_kde, hist_AP, plot_SAP_MAP

import tensorflow as tf

from sklearn.model_selection import train_test_split
import glob
import json
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import seaborn as sns
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

np.random.seed(2)
tf.random.set_seed(2)

In [5]:
all_pigs = ["P_{0:02d}_PulHyp".format(i) for i in range(1, 11)]
print(all_pigs)

pig_test_s_dict = {
    "P_01_PulHyp": 10338,
    "P_02_PulHyp": 12859,
    "P_03_PulHyp": 8124,
    "P_04_PulHyp": 8914,
    "P_05_PulHyp": 10063,
    "P_06_PulHyp": 6934,
    "P_07_PulHyp": 5692,
    "P_08_PulHyp": 7189,
    "P_09_PulHyp": 8228,
    "P_10_PulHyp": 6552,
}

load_path = "/data/PulHypStudie_Check_npz_v5_SNR20/"

['P_01_PulHyp', 'P_02_PulHyp', 'P_03_PulHyp', 'P_04_PulHyp', 'P_05_PulHyp', 'P_06_PulHyp', 'P_07_PulHyp', 'P_08_PulHyp', 'P_09_PulHyp', 'P_10_PulHyp']


In [6]:
dap_factor = 120
sap_factor = 180
map_factor = 150

import keras_tuner as kt
from tensorflow import keras
import pandas as pd
import seaborn as sns


def build_model(hp):
    output_dim = 3

    model = keras.Sequential()
    # input layer
    model.add(keras.layers.Input(shape=(64, 1024, 1)))

    # tune the number of hidden layers and units in each.
    for i in range(1, hp.Int("num_layers", 3, 7)):
        print(f"Init layer {i=}")
        hp_units = hp.Int("units_" + str(i), min_value=2, max_value=16, step=4)
        hp_kernel = hp.Int("kernel_" + str(i), min_value=2, max_value=14, step=1)
        # stride dim (0,1)
        hp_strides_0 = hp.Int("units_0_" + str(i), min_value=1, max_value=4, step=1)
        hp_strides_1 = hp.Int("units_1_" + str(i), min_value=2, max_value=4, step=1)
        hp_activation = hp.Choice(
            "activation_" + str(i), values=["relu", "elu", "tanh"]
        )
        hp_dropout = hp.Float("dropout_" + str(i), 0, 1.0, step=0.1)

        # create layer
        model.add(
            keras.layers.Conv2D(
                hp_units,
                hp_kernel,
                strides=(hp_strides_0, hp_strides_1),
                padding="same",
            )
        )
        model.add(keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation(hp_activation))
        model.add(keras.layers.Dropout(hp_dropout))

    model.add(keras.layers.Flatten())
    # output layer.
    model.add(keras.layers.Dense(output_dim, activation="linear"))

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4, 1e-5])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.MeanAbsoluteError(),
        # loss=keras.losses.MeanSquaredError(),
        metrics=["accuracy"],
    )
    print(model.summary())

    return model


tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=50,
    factor=2,
    directory="mapper_tuning_v5",
    project_name="mapper_tuning_20db",
)

Reloading Tuner from mapper_tuning_v5/mapper_tuning_20db/tuner0.json


In [7]:
LOOCV_nr = 10

In [ ]:
for p_n, test_pig in enumerate(all_pigs):
    print(f"Train all pics and test on {test_pig}")
    curr_pigs = ["P_{0:02d}_PulHyp".format(i) for i in range(1, 11)]
    del curr_pigs[p_n]
    print(curr_pigs)
    X_train, y_train, clrs_pig_train = load_augmented_example(
        load_path, curr_pigs, sample_skip=500, load_samples="upwards", shuffle=True
    )

    X_valid, y_valid, clrs_pig_valid = load_augmented_example(
        load_path, curr_pigs, sample_skip=500, load_samples="downwards", shuffle=True
    )
    X_test, Y_true, clrs_pig_test = load_augmented_example(
        load_path,
        [test_pig],
        sample_skip=pig_test_s_dict[test_pig],
        load_samples="downwards",
        shuffle=False,
    )

    print(
        X_train.shape,
        X_valid.shape,
        y_train.shape,
        y_valid.shape,
        X_test.shape,
        Y_true.shape,
        clrs_pig_train.shape,
        clrs_pig_valid.shape,
    )

    y_train[:, 0] = y_train[:, 0] / dap_factor  # dap normalization
    y_train[:, 1] = y_train[:, 1] / sap_factor  # sap normalization
    y_train[:, 2] = y_train[:, 2] / map_factor  # map normalization

    y_valid[:, 0] = y_valid[:, 0] / dap_factor  # dap normalization
    y_valid[:, 1] = y_valid[:, 1] / sap_factor  # sap normalization
    y_valid[:, 2] = y_valid[:, 2] / map_factor  # map normalization

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)

    history = model.fit(
        X_train,
        y_train,
        epochs=50,
        batch_size=8,
        validation_data=(X_valid, y_valid),
    )

    # predict EIT data of test pig
    y_pred = model.predict(X_test)
    Y_pred = np.empty(y_pred.shape)
    Y_pred[:, 0] = y_pred[:, 0] * dap_factor  # dap normalization
    Y_pred[:, 1] = y_pred[:, 1] * sap_factor  # sap normalization
    Y_pred[:, 2] = y_pred[:, 2] * map_factor  # map normalization

    np.savez(
        f"LOOCV{LOOCV_nr}/test_{test_pig}.npz",
        history=history.history,
        Y_pred=Y_pred,
        Y_true=Y_true,
    )

    model.save_weights(f"LOOCV{LOOCV_nr}/model_{test_pig}.weights.h5")

    DAP_err = Y_pred[:, 0] - Y_true[:, 0]
    SAP_err = Y_pred[:, 1] - Y_true[:, 1]
    MAP_err = Y_pred[:, 2] - Y_true[:, 2]
    DF_err = pd.DataFrame({"DAP": DAP_err, "SAP": SAP_err, "MAP": MAP_err})
    DF_err.to_csv(f"LOOCV{LOOCV_nr}/err_{test_pig}.csv", index=False)
    sns.histplot(DF_err)
    plt.savefig(f"LOOCV{LOOCV_nr}/histplot_{test_pig}.png")
    plt.show()

    sns.boxplot(DF_err)
    plt.grid()
    plt.ylim([-50, 50])
    plt.savefig(f"LOOCV{LOOCV_nr}/boxplot_{test_pig}.png")
    plt.show()

    plt.plot(Y_pred[:, 0], label="Pred")
    plt.plot(Y_true[:, 0], label="True")
    plt.legend()
    plt.grid()
    plt.savefig(f"LOOCV{LOOCV_nr}/boxplot_{test_pig}.png")
    plt.show()

Train all pics and test on P_01_PulHyp
['P_02_PulHyp', 'P_03_PulHyp', 'P_04_PulHyp', 'P_05_PulHyp', 'P_06_PulHyp', 'P_07_PulHyp', 'P_08_PulHyp', 'P_09_PulHyp', 'P_10_PulHyp']
Selected 14500 from /data/PulHypStudie_Check_npz_v5_SNR20/P_02_PulHyp/sample_000500.npz to /data/PulHypStudie_Check_npz_v5_SNR20/P_02_PulHyp/sample_014999.npz from pig P_02_PulHyp.
